# wrangling with the csv

In [1]:
import pandas as pd
import numpy as np
# import statsmodels.discrete.discrete_model as sm
import statsmodels as stt
import scipy.stats as sst
import os.path as osp

In [2]:
from statsmodels import api as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pwd

'/home/jb/code/repronim/simple2/simple2_analysis'

In [4]:
relative_path_filename = './data/blazegraph-query-all.csv'
assert osp.exists(relative_path_filename)

In [5]:
hie = pd.read_csv(relative_path_filename, na_values='nd')
original_col_names = list(hie)
print(original_col_names)
assert len(original_col_names) == len(set(original_col_names))

['study', 'ID', 'Age', 'Gender', 'diagnostic', 'value', 'tool', 'structure']


In [8]:
uberon = "http://purl.obolibrary.org/obo/UBERON_0001897"

cmd = 'wget -O - ' + uberon\
                   + '  | grep -A 1 ' +  "'" + 'Class rdf:about='\
                   + '"' + uberon + '"' +  "'"

In [9]:
print(cmd.split())

['wget', '-O', '-', 'http://purl.obolibrary.org/obo/UBERON_0001897', '|', 'grep', '-A', '1', "'Class", 'rdf:about="http://purl.obolibrary.org/obo/UBERON_0001897"\'']


In [10]:
import subprocess
MyOut = subprocess.Popen(cmd.split(), 
            stdout=subprocess.PIPE, 
           stderr=subprocess.STDOUT, shell=True)
stdout,stderr = MyOut.communicate()
print(stdout)
#print(stderr)

b"wget: missing URL\nUsage: wget [OPTION]... [URL]...\n\nTry `wget --help' for more options.\n"


In [11]:
process = subprocess.run(cmd.split())

In [12]:
uberon

'http://purl.obolibrary.org/obo/UBERON_0001897'

In [24]:
set(hie.structure)

{'<http://purl.obolibrary.org/obo/UBERON_0000959>',
 '<http://purl.obolibrary.org/obo/UBERON_0001869>',
 '<http://purl.obolibrary.org/obo/UBERON_0001873>',
 '<http://purl.obolibrary.org/obo/UBERON_0001874>',
 '<http://purl.obolibrary.org/obo/UBERON_0001876>',
 '<http://purl.obolibrary.org/obo/UBERON_0001882>',
 '<http://purl.obolibrary.org/obo/UBERON_0001886>',
 '<http://purl.obolibrary.org/obo/UBERON_0001897>',
 '<http://purl.obolibrary.org/obo/UBERON_0001954>',
 '<http://purl.obolibrary.org/obo/UBERON_0002037>',
 '<http://purl.obolibrary.org/obo/UBERON_0002129>',
 '<http://purl.obolibrary.org/obo/UBERON_0002285>',
 '<http://purl.obolibrary.org/obo/UBERON_0002286>',
 '<http://purl.obolibrary.org/obo/UBERON_0002298>',
 '<http://purl.obolibrary.org/obo/UBERON_0002422>',
 '<http://purl.obolibrary.org/obo/UBERON_0006091>',
 '<http://purl.obolibrary.org/obo/UBERON_0006514>',
 '<http://purl.obolibrary.org/obo/UBERON_0009857>',
 '<http://purl.obolibrary.org/obo/UBERON_0014930>'}

In [13]:
list(hie)

['study', 'ID', 'Age', 'Gender', 'diagnostic', 'value', 'tool', 'structure']

In [25]:
uberon= '<http://purl.obolibrary.org/obo/UBERON_0001897>'
tmp = hie[hie.structure==uberon]

In [27]:
len(tmp)

5375

In [ ]:
# Load in the dataset, appending 'nd' to the list of possible na_values
hie = pd.read_csv(relative_path_filename, na_values='nd')
original_col_names = list(hie)
print(original_col_names[:10])

# column names are unique
assert len(original_col_names) == len(set(original_col_names))

In [ ]:
def get_structure_idx(ilx, dataframe):
    """
    get indexes of the column names that contain ilx
    """
    col_names = list(dataframe)
    idxs = [idx for (idx, value) in enumerate(col_names) if ilx in value]
    return(idxs)

In [ ]:
def get_structure_col_names(ilx, dataframe):
    """
    get column names that contain ilx
    """
    col_names = list(dataframe)
    cnames = [cname for cname in col_names if ilx in cname]
    return(cnames)

In [ ]:
get_structure_col_names('ILX:0381415', hie)

In [ ]:
ilx_bases = [ilx for ilx in original_col_names if  '.' not in ilx 
                                              and ilx[:4] == 'ILX:']
print(len(ilx_bases))

In [ ]:
structures_names = [get_structure_col_names(ilx, hie) for ilx in ilx_bases]

In [ ]:
print(structures_names)

In [ ]:
[hie[roi][1] for roi in structures_names[0]]

In [ ]:
def get_data_for_regions(regions, dataframe):
    """
    
    """
    col_names = list(dataframe)
    return np.asarray( [ [float(x) for x in dataframe[roi][4:]] for roi in regions])

In [ ]:
rois = structures_names[1]
print(rois)
data = get_data_for_regions(rois, hie)
data.shape

In [ ]:
# remove nan lines
data_not_nan = ~np.isnan(data)
to_keep = np.all(data_not_nan, axis=0)
data_ok = data[:,to_keep]

In [ ]:
data.shape
data_ok.shape

In [ ]:
a = plt.plot(data_ok.T)

In [ ]:
roi = l3rois[0]
hie[roi[0]][0]

In [ ]:
cc = np.corrcoef(data_ok)
cc.shape

In [ ]:
l3rois = [rois for rois in structures_names if len(rois) == 3]
correlations = []
sample_size = []
roi_name = []

for rois in l3rois:
    roi_name.append(hie[rois[0]][0])
    data = get_data_for_regions(rois, hie)
    data_not_nan = ~np.isnan(data)
    data_ok = data[:,np.all(data_not_nan, axis=0)]
    sample_size.append(data_ok.shape[1])
    cc = np.corrcoef(data_ok)
    correlations.append([cc[0,1], cc[1,2], cc[0,2]])
    

In [ ]:
for roi, ccoeff in zip(roi_name, correlations):
    print("{0:<35}:".format(roi) + " {:.3} {:.3} {:.3}".format(ccoeff[0], ccoeff[1], ccoeff[2]))

    